In [4]:
import pandas as pd
import numpy as np

def preprocess_flood_data(df):
    # Create a copy to avoid modifying original data
    processed_df = df.copy()
    
    # Fill empty flood values with 0
    processed_df['Flood?'] = processed_df['Flood?'].fillna(0)
    
    # Convert Flood? column to integer type
    processed_df['Flood?'] = processed_df['Flood?'].astype(int)
    
    # Select relevant meteorological columns
    relevant_columns = [
        'Max_Temp',      # Maximum temperature
        'Min_Temp',      # Minimum temperature
        'Rainfall',      # Rainfall amount
        'Relative_Humidity',  # Humidity
        'Wind_Speed',    # Wind speed
        'Cloud_Coverage',  # Cloud coverage
        'Month',         # Month for seasonal patterns
        'Flood?'         # Target variable
    ]
    
    # Create final dataset with selected columns
    final_df = processed_df[relevant_columns]
    
    return final_df

# Load the data
df = pd.read_csv('Balanced_dataset2.csv')  # Replace with your actual file path

# Process the data
processed_df = preprocess_flood_data(df)

# Display basic statistics about the processed dataset
print("Dataset Statistics:")
print(f"Total samples: {len(processed_df)}")
print(f"Flood events: {processed_df['Flood?'].sum()} ({(processed_df['Flood?'].mean()*100):.2f}%)")

print("\nFeature Statistics:")
for col in processed_df.columns:
    if col != 'Flood?':
        print(f"\n{col}:")
        print(f"Mean: {processed_df[col].mean():.2f}")
        print(f"Std: {processed_df[col].std():.2f}")
        print(f"Min: {processed_df[col].min():.2f}")
        print(f"Max: {processed_df[col].max():.2f}")
        print(f"Correlation with floods: {processed_df[col].corr(processed_df['Flood?']):.3f}")

# Save processed dataset
# processed_df.to_csv('processed_flood_data.csv', index=False)

Dataset Statistics:
Total samples: 12132
Flood events: 4132 (34.06%)

Feature Statistics:

Max_Temp:
Mean: 33.59
Std: 2.78
Min: 24.60
Max: 44.00
Correlation with floods: 0.172

Min_Temp:
Mean: 21.88
Std: 4.79
Min: 6.50
Max: 27.80
Correlation with floods: 0.503

Rainfall:
Mean: 264.33
Std: 278.02
Min: 0.00
Max: 2072.00
Correlation with floods: 0.784

Relative_Humidity:
Mean: 80.78
Std: 7.51
Min: 34.00
Max: 96.00
Correlation with floods: 0.550

Wind_Speed:
Mean: 1.49
Std: 1.08
Min: 0.00
Max: 11.20
Correlation with floods: 0.248

Cloud_Coverage:
Mean: 3.91
Std: 2.13
Min: 0.00
Max: 7.90
Correlation with floods: 0.660

Month:
Mean: 6.62
Std: 3.20
Min: 1.00
Max: 12.00
Correlation with floods: 0.136


In [ ]:
# import numpy as np
# import pandas as pd
# import torch
# from torch import nn
# import torch.nn.functional as F
# from torch.utils.data import DataLoader, TensorDataset
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split

# class FloodPredictor:
#     def __init__(self, use_gpu=True):
#         self.device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
        
#         # Updated realistic bounds based on your dataset statistics
#         self.realistic_bounds = {
#             'Max_Temp': (15, 45),      # Realistic range for max temperature in Celsius
#             'Min_Temp': (5, 30),       # Realistic range for min temperature in Celsius
#             'Rainfall': (0, 500),      # Maximum daily rainfall in mm
#             'Relative_Humidity': (30, 100),  # Humidity percentage
#             'Wind_Speed': (0, 20),     # Wind speed in m/s
#             'Cloud_Coverage': (0, 1),   # Cloud coverage as fraction
#             'Month': (1, 12)           # Month of the year
#         }
        
#         # Initialize thresholds with multiple levels
#         self.thresholds = {k: {
#             'low': torch.tensor(v[0] + (v[1] - v[0])*0.3, device=self.device),
#             'medium': torch.tensor(v[0] + (v[1] - v[0])*0.5, device=self.device),
#             'high': torch.tensor(v[0] + (v[1] - v[0])*0.7, device=self.device)
#         } for k, v in self.realistic_bounds.items()}
        
#         # Adjusted deltas for each feature
#         self.delta = {
#             'Max_Temp': 1.0,           # 1°C change
#             'Min_Temp': 1.0,           # 1°C change
#             'Rainfall': 10.0,          # 10mm change
#             'Relative_Humidity': 5.0,   # 5% change
#             'Wind_Speed': 0.5,         # 0.5 m/s change
#             'Cloud_Coverage': 0.1,      # 0.1 fraction change
#             'Month': 1.0               # 1 month change
#         }
        
#         self.num_bins = 20
#         self.num_actions = 3
        
#         # Create Q-tables for each feature and threshold level
#         self.Q_tables = {
#             feature: {
#                 level: torch.zeros((self.num_bins, self.num_actions), device=self.device)
#                 for level in ['low', 'medium', 'high']
#             } for feature in self.realistic_bounds.keys()
#         }
        
#         self.history = {
#             'accuracy': [],
#             'thresholds': []
#         }

#     def compute_feature_interactions(self, batch_data):
#         interactions = {}
        
#         # Rainfall and humidity interaction
#         interactions['rainfall_humidity'] = (
#             batch_data['Rainfall'] * 
#             batch_data['Relative_Humidity'] / 100
#         )
        
#         # Temperature range interaction
#         interactions['temp_range'] = (
#             batch_data['Max_Temp'] - 
#             batch_data['Min_Temp']
#         )
        
#         # Rainfall and wind interaction (can affect flood severity)
#         interactions['rainfall_wind'] = (
#             batch_data['Rainfall'] * 
#             batch_data['Wind_Speed']
#         )
        
#         # Cloud and humidity interaction
#         interactions['cloud_humidity'] = (
#             batch_data['Cloud_Coverage'] * 
#             batch_data['Relative_Humidity']
#         )
        
#         return interactions

#     def rule_based_flood_probability(self, batch_data):
#         batch_size = batch_data['Rainfall'].shape[0]
#         scores = torch.zeros(batch_size, device=self.device)
        
#         # Compute feature interactions
#         interactions = self.compute_feature_interactions(batch_data)
        
#         # Basic feature scoring with multiple thresholds
#         # Rainfall has highest weight
#         feature_weights = {
#             'Rainfall': 3.0,
#             'Relative_Humidity': 1.5,
#             'Wind_Speed': 1.0,
#             'Cloud_Coverage': 1.0,
#             'Max_Temp': 0.5,
#             'Min_Temp': 0.5,
#             'Month': 0.5  # Seasonal effect
#         }
        
#         for feature, weight in feature_weights.items():
#             feature_data = batch_data[feature]
#             scores += weight * torch.where(
#                 feature_data > self.thresholds[feature]['high'], 3,
#                 torch.where(feature_data > self.thresholds[feature]['medium'], 2,
#                 torch.where(feature_data > self.thresholds[feature]['low'], 1, 
#                 torch.zeros(batch_size, device=self.device)))
#             )
        
#         # Add interaction scores
#         scores += torch.where(
#             interactions['rainfall_humidity'] > self.compute_adaptive_threshold(interactions['rainfall_humidity']),
#             2 * torch.ones(batch_size, device=self.device),
#             torch.zeros(batch_size, device=self.device)
#         )
        
#         scores += torch.where(
#             interactions['rainfall_wind'] > self.compute_adaptive_threshold(interactions['rainfall_wind']),
#             1.5 * torch.ones(batch_size, device=self.device),
#             torch.zeros(batch_size, device=self.device)
#         )
        
#         # Normalize scores
#         max_possible_score = 30  # Adjusted based on new weights
#         return torch.clamp(scores / max_possible_score, 0, 1)

#     def prepare_batch_data(self, df, batch_indices):
#         return {
#             col: torch.tensor(df[col].values[batch_indices], 
#                             dtype=torch.float32, 
#                             device=self.device)
#             for col in df.columns if col != 'Flood?'
#         }

#     # Other methods remain the same as in the original code
#     def discretize_threshold(self, value, min_val, max_val):
#         value = torch.tensor(value, device=self.device) if not torch.is_tensor(value) else value
#         value = torch.clamp(value, min_val, max_val)
#         bin_width = (max_val - min_val) / self.num_bins
#         state = ((value - min_val) / bin_width).long()
#         return torch.clamp(state, 0, self.num_bins - 1)

#     def compute_adaptive_threshold(self, values):
#         return values.mean() + values.std()

#     def compute_accuracy(self, batch_data):
#         probs = self.rule_based_flood_probability(batch_data)
#         predictions = (probs > 0.5).float()
#         return (predictions == batch_data['Flood?']).float().mean()

#     def train(self, df, num_episodes=100000, batch_size=128, alpha=0.1, gamma=0.9, epsilon=0.1):
#         train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
        
#         best_accuracy = 0
#         best_thresholds = None
#         patience = 1000
#         no_improve_count = 0
        
#         for episode in range(num_episodes):
#             batch_indices = np.random.choice(len(train_df), batch_size)
#             batch_data = self.prepare_batch_data(train_df, batch_indices)
#             batch_data['Flood?'] = torch.tensor(
#                 train_df['Flood?'].values[batch_indices],
#                 dtype=torch.float32,
#                 device=self.device
#             )
            
#             episode_baseline = self.compute_accuracy(batch_data)
            
#             # Update thresholds for each feature and level
#             for feature in self.thresholds:
#                 min_val, max_val = self.realistic_bounds[feature]
#                 for level in ['low', 'medium', 'high']:
#                     current_threshold = self.thresholds[feature][level]
#                     state = self.discretize_threshold(current_threshold, min_val, max_val)
                    
#                     if torch.rand(1, device=self.device) < epsilon:
#                         action_idx = torch.randint(0, self.num_actions, (1,), device=self.device)
#                     else:
#                         action_idx = self.Q_tables[feature][level][state].argmax()
                    
#                     actions = torch.tensor([-1, 0, 1], device=self.device)
#                     new_threshold = current_threshold + actions[action_idx] * self.delta[feature]
#                     self.thresholds[feature][level] = torch.clamp(new_threshold, min_val, max_val)
                    
#                     new_accuracy = self.compute_accuracy(batch_data)
#                     reward = new_accuracy - episode_baseline
#                     new_state = self.discretize_threshold(self.thresholds[feature][level], min_val, max_val)
                    
#                     self.Q_tables[feature][level][state, action_idx] += alpha * (
#                         reward + gamma * self.Q_tables[feature][level][new_state].max() -
#                         self.Q_tables[feature][level][state, action_idx]
#                     )
            
#             if (episode + 1) % 100 == 0:
#                 val_data = self.prepare_batch_data(val_df, np.arange(len(val_df)))
#                 val_data['Flood?'] = torch.tensor(
#                     val_df['Flood?'].values,
#                     dtype=torch.float32,
#                     device=self.device
#                 )
#                 val_accuracy = self.compute_accuracy(val_data)
                
#                 self.history['accuracy'].append(val_accuracy.item())
#                 self.history['thresholds'].append(
#                     {k: {kk: vv.item() for kk, vv in v.items()} 
#                      for k, v in self.thresholds.items()}
#                 )
                
#                 if val_accuracy > best_accuracy:
#                     best_accuracy = val_accuracy
#                     best_thresholds = {k: {kk: vv.clone() for kk, vv in v.items()} 
#                                      for k, v in self.thresholds.items()}
#                     no_improve_count = 0
#                 else:
#                     no_improve_count += 1
                
#                 if no_improve_count >= patience:
#                     print(f"Early stopping at episode {episode+1}")
#                     self.thresholds = best_thresholds
#                     break
                
#                 if (episode + 1) % 10 == 0:
#                     print(f"Episode {episode+1}, Train Accuracy: {episode_baseline:.4f}, "
#                           f"Val Accuracy: {val_accuracy:.4f}")
#                     print(self.history['thresholds'][-1])

# # Usage example
# if __name__ == "__main__":
#     df = pd.read_csv('Balanced_dataset.csv')
#     processed_df = preprocess_flood_data(df)
    
#     # Create and train model
#     model = FloodPredictor(use_gpu=True)
#     model.train(processed_df, num_episodes=20000, batch_size=128)

Episode 100, Train Accuracy: 0.9375, Val Accuracy: 0.8832
{'Max_Temp': {'low': 15.0, 'medium': 15.0, 'high': 15.0}, 'Min_Temp': {'low': 5.0, 'medium': 5.0, 'high': 5.0}, 'Rainfall': {'low': 30.0, 'medium': 290.0, 'high': 370.0}, 'Relative_Humidity': {'low': 35.0, 'medium': 30.0, 'high': 40.0}, 'Wind_Speed': {'low': 5.5, 'medium': 2.5, 'high': 11.0}, 'Cloud_Coverage': {'low': 0.30000001192092896, 'medium': 0.10000000149011612, 'high': 0.4000000059604645}, 'Month': {'low': 3.0, 'medium': 3.0, 'high': 4.0}}
Episode 200, Train Accuracy: 0.8750, Val Accuracy: 0.9068
{'Max_Temp': {'low': 15.0, 'medium': 15.0, 'high': 16.0}, 'Min_Temp': {'low': 5.0, 'medium': 5.0, 'high': 5.0}, 'Rainfall': {'low': 50.0, 'medium': 320.0, 'high': 400.0}, 'Relative_Humidity': {'low': 35.0, 'medium': 40.0, 'high': 30.0}, 'Wind_Speed': {'low': 9.0, 'medium': 7.0, 'high': 3.5}, 'Cloud_Coverage': {'low': 0.10000001639127731, 'medium': 0.30000001192092896, 'high': 0.20000001788139343}, 'Month': {'low': 1.0, 'medium':

In [5]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def preprocess_flood_data(df):
    processed_df = df.copy()
    processed_df['Flood?'] = processed_df['Flood?'].fillna(0)
    processed_df['Flood?'] = processed_df['Flood?'].astype(int)
    relevant_columns = [
        'Max_Temp', 'Min_Temp', 'Rainfall', 
        'Relative_Humidity', 'Wind_Speed', 'Cloud_Coverage', 
        'Month', 'Flood?'
    ]
    final_df = processed_df[relevant_columns]
    return final_df

class FloodPredictor:
    def __init__(self, use_gpu=True):
        self.device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')
        
        # Updated realistic bounds (you may adjust these based on your data)
        self.realistic_bounds = {
            'Max_Temp': (25, 44),      
            'Min_Temp': (6, 28),       
            'Rainfall': (0, 2072),
            'Relative_Humidity': (30, 100),
            'Wind_Speed': (0, 20),     
            'Cloud_Coverage': (0, 1),
            # 'Month': (1, 12)
        }
        
        self.thresholds = {k: {
            'low': torch.tensor(v[0] + (v[1] - v[0]) * 0.3, device=self.device),
            'medium': torch.tensor(v[0] + (v[1] - v[0]) * 0.5, device=self.device),
            'high': torch.tensor(v[0] + (v[1] - v[0]) * 0.7, device=self.device)
        } for k, v in self.realistic_bounds.items()}
        
        self.delta = {
            'Max_Temp': 1.0,
            'Min_Temp': 1.0,
            'Rainfall': 10.0,
            'Relative_Humidity': 5.0,
            'Wind_Speed': 0.5,
            'Cloud_Coverage': 0.1,
        }
        
        self.num_bins = 20
        self.num_actions = 3
        self.Q_tables = {
            feature: {
                level: torch.zeros((self.num_bins, self.num_actions), device=self.device)
                for level in ['low', 'medium', 'high']
            } for feature in self.realistic_bounds.keys()
        }
        
        self.history = {
            'accuracy': [],
            'thresholds': []
        }
    def enforce_threshold_order(self, feature):
        """Ensure thresholds maintain low < medium < high order"""
        min_val = torch.tensor(self.realistic_bounds[feature][0], 
                              dtype=torch.float32, 
                              device=self.device)
        max_val = torch.tensor(self.realistic_bounds[feature][1], 
                              dtype=torch.float32, 
                              device=self.device)

        # Clamp low threshold between min_val and medium
        self.thresholds[feature]['low'] = torch.clamp(
            self.thresholds[feature]['low'], 
            min_val,
            self.thresholds[feature]['medium']
        )

        # Clamp medium threshold between low and high
        self.thresholds[feature]['medium'] = torch.clamp(
            self.thresholds[feature]['medium'],
            self.thresholds[feature]['low'],
            self.thresholds[feature]['high']
        )
    
        # Clamp high threshold between medium and max_val
        self.thresholds[feature]['high'] = torch.clamp(
            self.thresholds[feature]['high'],
            self.thresholds[feature]['medium'],
            max_val
        )
    def smooth_score(self, feature_data, thresholds_dict, weight, k=0.05):
        """
        Computes a smooth score using a sigmoid function around each threshold.
        """
        s_low = torch.sigmoid(k * (feature_data - thresholds_dict['low']))
        s_med = torch.sigmoid(k * (feature_data - thresholds_dict['medium']))
        s_high = torch.sigmoid(k * (feature_data - thresholds_dict['high']))
        return weight * (s_low + s_med + s_high)
    
    def compute_feature_interactions(self, batch_data):
        interactions = {}
        interactions['rainfall_humidity'] = (batch_data['Rainfall'] * batch_data['Relative_Humidity'] / 100)
        interactions['temp_range'] = (batch_data['Max_Temp'] - batch_data['Min_Temp'])
        interactions['rainfall_wind'] = (batch_data['Rainfall'] * batch_data['Wind_Speed'])
        interactions['cloud_humidity'] = (batch_data['Cloud_Coverage'] * batch_data['Relative_Humidity'])
        return interactions
    def normalize_feature(self, feature_data, feature_name):
        min_val, max_val = self.realistic_bounds[feature_name]
        return (feature_data - min_val) / (max_val - min_val)

    def rule_based_flood_probability(self, batch_data):
        batch_size = batch_data['Rainfall'].shape[0]
        scores = torch.zeros(batch_size, device=self.device)
        normalized_data = {
        feature: self.normalize_feature(data, feature)
        for feature, data in batch_data.items()
        if feature in self.realistic_bounds
    }
        interactions = self.compute_feature_interactions(normalized_data)
        
        # Define weights for each feature
        feature_weights = {
            'Rainfall': 4.0,
            'Relative_Humidity': 2.0,
            'Wind_Speed': 1.0,
            'Cloud_Coverage': 0.8,
            'Max_Temp': 0.5,
            'Min_Temp': 2.0,
        }
        
        for feature, weight in feature_weights.items():
            scores += self.smooth_score(batch_data[feature], self.thresholds[feature], weight, k=0.05)
        
        scores += torch.where(
            interactions['rainfall_humidity'] > self.compute_adaptive_threshold(interactions['rainfall_humidity']),
            2 * torch.ones(batch_size, device=self.device),
            torch.zeros(batch_size, device=self.device)
        )
        scores += torch.where(
            interactions['rainfall_wind'] > self.compute_adaptive_threshold(interactions['rainfall_wind']),
            1.5 * torch.ones(batch_size, device=self.device),
            torch.zeros(batch_size, device=self.device)
        )
        
        max_possible_score = 30
        return torch.clamp(scores / max_possible_score, 0, 1)
    
    def prepare_batch_data(self, df, batch_indices):
        return {
            col: torch.tensor(df[col].values[batch_indices], 
                            dtype=torch.float32, 
                            device=self.device)
            for col in df.columns if col != 'Flood?'
        }
    
    def discretize_threshold(self, value, min_val, max_val):
        value = torch.tensor(value, device=self.device) if not torch.is_tensor(value) else value
        value = torch.clamp(value, min_val, max_val)
        bin_width = (max_val - min_val) / self.num_bins
        state = ((value - min_val) / bin_width).long()
        return torch.clamp(state, 0, self.num_bins - 1)
    
    def compute_adaptive_threshold(self, values):
        if values.numel() > 1:
            return values.mean() + values.std()
        else:
            return values.mean()
    
    def compute_accuracy(self, batch_data):
        probs = self.rule_based_flood_probability(batch_data)
        predictions = (probs > 0.5).float()
        return (predictions == batch_data['Flood?']).float().mean()
    def validate_prediction(self, features, prediction, actual=None):
    # Convert tensor values to float for comparison
        rainfall = features['Rainfall'].item()
        humidity = features['Relative_Humidity'].item()
        min_temp = features['Min_Temp'].item()

        # Check if prediction makes sense given the features
        high_risk = (
            rainfall > 500 and 
            humidity > 70 and 
            min_temp > 20
        )
        low_risk = (
            rainfall < 100 and 
            humidity < 50
        )

        if high_risk and prediction < 0.5:
            print("Warning: Predicted no flood despite high-risk conditions")
        elif low_risk and prediction > 0.5:
            print("Warning: Predicted flood despite low-risk conditions")

        if actual is not None:
            print(f"Predicted: {prediction:.3f}, Actual: {actual}")
            # Convert tensor values to float before formatting
            formatted_features = {
                k: f"{v.item():.2f}" if torch.is_tensor(v) else f"{v:.2f}" 
                for k, v in features.items()
            }
            print("Features:", formatted_features)
    def train(self, df, num_episodes=100000, batch_size=128, alpha=0.1, gamma=0.9, initial_epsilon=0.1):
        train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
        
        best_accuracy = 0
        best_thresholds = None
        patience = 1000
        no_improve_count = 0
        min_epsilon = 0.01
        decay_rate = 0.9995
        
        for episode in range(num_episodes):
            current_epsilon = max(initial_epsilon * (decay_rate ** episode), min_epsilon)
            
            batch_indices = np.random.choice(len(train_df), batch_size)
            batch_data = self.prepare_batch_data(train_df, batch_indices)
            batch_data['Flood?'] = torch.tensor(
                train_df['Flood?'].values[batch_indices],
                dtype=torch.float32,
                device=self.device
            )
            
            episode_baseline = self.compute_accuracy(batch_data)
            
            for feature in self.thresholds:
                min_val, max_val = self.realistic_bounds[feature]
                for level in ['low', 'medium', 'high']:
                    current_threshold = self.thresholds[feature][level]
                    state = self.discretize_threshold(current_threshold, min_val, max_val)
                    
                    if torch.rand(1, device=self.device) < current_epsilon:
                        action_idx = torch.randint(0, self.num_actions, (1,), device=self.device)
                    else:
                        action_idx = self.Q_tables[feature][level][state].argmax()
                    
                    actions_tensor = torch.tensor([-1, 0, 1], device=self.device)
                    new_threshold = current_threshold + actions_tensor[action_idx] * self.delta[feature]
                    self.thresholds[feature][level] = torch.clamp(new_threshold, min_val, max_val)
                    
                    new_accuracy = self.compute_accuracy(batch_data)
                    reward = new_accuracy - episode_baseline
                    if feature == 'Rainfall' and self.thresholds[feature][level] < 50:
                        reward -= 5
                    new_state = self.discretize_threshold(self.thresholds[feature][level], min_val, max_val)
                    
                    self.Q_tables[feature][level][state, action_idx] += alpha * (
                        reward + gamma * self.Q_tables[feature][level][new_state].max() -
                        self.Q_tables[feature][level][state, action_idx]
                    )
            for feature in self.thresholds:
                self.enforce_threshold_order(feature)
                
            if (episode + 1) % 1000 == 0:
                val_data = self.prepare_batch_data(val_df, np.arange(len(val_df)))
                val_data['Flood?'] = torch.tensor(
                    val_df['Flood?'].values,
                    dtype=torch.float32,
                    device=self.device
                )
                val_accuracy = self.compute_accuracy(val_data)
                
                self.history['accuracy'].append(val_accuracy.item())
                self.history['thresholds'].append(
                    {k: {kk: vv.item() for kk, vv in v.items()} 
                     for k, v in self.thresholds.items()}
                )
                
                if val_accuracy > best_accuracy:
                    best_accuracy = val_accuracy
                    best_thresholds = {k: {kk: vv.clone() for kk, vv in v.items()} 
                                     for k, v in self.thresholds.items()}
                    no_improve_count = 0
                else:
                    no_improve_count += 1
                
                if no_improve_count >= patience:
                    print(f"Early stopping at episode {episode+1}")
                    self.thresholds = best_thresholds
                    break
                
                print(f"Episode {episode+1}, Train Accuracy: {episode_baseline:.4f}, "
                      f"Val Accuracy: {val_accuracy:.4f}")
                print(self.history['thresholds'][-1])

# if __name__ == "__main__":
#     df = pd.read_csv('Balanced_dataset2.csv')
#     processed_df = preprocess_flood_data(df)
    
#     model = FloodPredictor(use_gpu=True)
#     model.train(processed_df, num_episodes=10000, batch_size=128)
#     test_data = {
#     'Max_Temp': torch.tensor([34.3], dtype=torch.float32, device=model.device),
#     'Min_Temp': torch.tensor([25.6], dtype=torch.float32, device=model.device),
#     'Rainfall': torch.tensor([312], dtype=torch.float32, device=model.device),
#     'Relative_Humidity': torch.tensor([91], dtype=torch.float32, device=model.device),
#     'Wind_Speed': torch.tensor([0.9], dtype=torch.float32, device=model.device),
#     'Cloud_Coverage': torch.tensor([5.9], dtype=torch.float32, device=model.device),
#     'Month': torch.tensor([8], dtype=torch.float32, device=model.device)
# }

#     pred = model.rule_based_flood_probability(test_data)
#     model.validate_prediction(test_data, pred.item(), actual=1)


In [10]:
test_data = {
    'Max_Temp': torch.tensor([34.3], dtype=torch.float32, device=model.device),
    'Min_Temp': torch.tensor([25.6], dtype=torch.float32, device=model.device),
    'Rainfall': torch.tensor([312], dtype=torch.float32, device=model.device),
    'Relative_Humidity': torch.tensor([91], dtype=torch.float32, device=model.device),
    'Wind_Speed': torch.tensor([0.9], dtype=torch.float32, device=model.device),
    'Cloud_Coverage': torch.tensor([5.9], dtype=torch.float32, device=model.device),
    # 'Month': torch.tensor([8], dtype=torch.float32, device=model.device)
}

pred = model.rule_based_flood_probability(test_data)
model.validate_prediction(test_data, pred.item(), actual=1)

Predicted: 0.797, Actual: 1
Features: {'Max_Temp': '34.30', 'Min_Temp': '25.60', 'Rainfall': '312.00', 'Relative_Humidity': '91.00', 'Wind_Speed': '0.90', 'Cloud_Coverage': '5.90'}


In [1]:
test_data = {
    'Max_Temp': torch.tensor([37.8], dtype=torch.float32, device=model.device),
    'Min_Temp': torch.tensor([26], dtype=torch.float32, device=model.device),
    'Rainfall': torch.tensor([173], dtype=torch.float32, device=model.device),
    'Relative_Humidity': torch.tensor([86], dtype=torch.float32, device=model.device),
    'Wind_Speed': torch.tensor([1.7], dtype=torch.float32, device=model.device),
    'Cloud_Coverage': torch.tensor([5.2], dtype=torch.float32, device=model.device),
    # 'Month': torch.tensor([8], dtype=torch.float32, device=model.device)
}

pred = model.rule_based_flood_probability(test_data)
model.validate_prediction(test_data, pred.item(), actual=0)
# torch.save(model.thresholds, 'thresholds.pth')
# torch.save(model.Q_tables, 'Q_tables.pth')


NameError: name 'torch' is not defined

In [12]:
loaded_thresholds = torch.load('thresholds.pth', map_location=model.device)
loaded_Q_tables = torch.load('Q_tables.pth', map_location=model.device)

# Update your model's parameters
model.thresholds = loaded_thresholds
model.Q_tables = loaded_Q_tables

C:\Users\Karan\AppData\Local\Temp\ipykernel_12248\3024628763.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_thresholds = torch.load('thresholds.pth', map_locatio

In [9]:
import torch
model = FloodPredictor(use_gpu=True)
loaded_thresholds = torch.load('thresholds.pth', map_location=model.device)
loaded_Q_tables = torch.load('Q_tables.pth', map_location=model.device)
model.thresholds = loaded_thresholds
model.Q_tables = loaded_Q_tables
test_data = {
    'Max_Temp': torch.tensor([35.99], dtype=torch.float32, device=model.device),
    'Min_Temp': torch.tensor([30.94], dtype=torch.float32, device=model.device),
    'Rainfall': torch.tensor([0], dtype=torch.float32, device=model.device),
    'Relative_Humidity': torch.tensor([16], dtype=torch.float32, device=model.device),
    'Wind_Speed': torch.tensor([4.12], dtype=torch.float32, device=model.device),
    'Cloud_Coverage': torch.tensor([0], dtype=torch.float32, device=model.device),
    'Month': torch.tensor([2], dtype=torch.float32, device=model.device)
}

flood_probability = model.rule_based_flood_probability(test_data)
print("Predicted Flood Probability:", flood_probability.item())


Predicted Flood Probability: 0.3176373541355133


C:\Users\Karan\AppData\Local\Temp\ipykernel_40720\74721797.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_thresholds = torch.load('thresholds.pth', map_location=